<a href="https://colab.research.google.com/github/vamsikrishna00466/EDA/blob/main/st-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENTRY BUY PRICE**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df =pd.read_excel('/content/BankNiftyFutures_Data5Min.xlsx')

In [ ]:
def TrueRange(data):
    data = data.copy()
    data["TR"] = np.nan
    for i in range(1,len(data)):
        h = data.loc[i,"High 5"]
        l = data.loc[i,"Low 5"]
        pc = data.loc[i-1,"Close 5"]
        
        x = h-l
        y = abs(h-pc)
        z = abs(l-pc)
        
        TR = max(x,y,z)
        data.loc[i,"TR"] = TR
    
    return data

In [ ]:
new_df = TrueRange(df)

In [ ]:
def average_true_range(data, period=7, drop_tr=True, smoothing="SMA"):
    """
    Average True Range
    :param data: Pandas Dataframe
    :param period: Period for which the ATR needs to be calculated
    :param drop_tr: Whether to drop the TR field
    :param smoothing: Smoothing type - Possible values: RMA, SMA, EMA
    :return: Pandas Dataframe with new column atr_<period>_<smoothing>. Ex: atr_14_sma
    """
    data = data.copy()
    if smoothing == "RMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].ewm(com=period - 1,
                                                                           min_periods=period).mean()
    elif smoothing == "SMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].rolling(window=period).mean()
    elif smoothing == "EMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].ewm(span=period, adjust=False).mean()
    
    if drop_tr:
        data.drop(['TR'], inplace=True, axis=1)
    data = data.round(decimals=2)

    return data

In [ ]:
ATR_df = average_true_range(data = new_df, period=18, drop_tr=True, smoothing="SMA")


In [ ]:
df = ATR_df.copy()
f=3
n=19

In [ ]:

df['Upper Basic']=(df['High 5']+df['Low 5'])/2+(f*df['atr_18_SMA'])
df['Lower Basic']=(df['High 5']+df['Low 5'])/2-(f*df['atr_18_SMA'])
df['Upper Band']=df['Upper Basic']
df['Lower Band']=df['Lower Basic']

#super trend
df['SuperTrend']=np.nan
for i in df['SuperTrend']:
  if df['Close 5'].iloc[n-1]<=df['Upper Band'].iloc[n-1]:
    df['SuperTrend'].iloc[n-1]=df['Upper Band'].iloc[n-1]
  elif df['Close 5'].iloc[n-1]>df['Upper Band'].iloc[i]:
    df['SuperTrend'].iloc[n-1]=df['Lower Band'].iloc[n-1]
for i in range(n,len(df)):
  if df['SuperTrend'].iloc[i-1]==df['Upper Band'].iloc[i-1] and df['Close 5'].iloc[i]<=df['Upper Band'].iloc[i]:
    df['SuperTrend'].iloc[i]=df['Upper Band'].iloc[i]
  elif  df['SuperTrend'].iloc[i-1]==df['Upper Band'].iloc[i-1] and df['Close 5'].iloc[i]>=df['Upper Band'].iloc[i]:
    df['SuperTrend'].iloc[i]=df['Lower Band'].iloc[i]
  elif df['SuperTrend'].iloc[i-1]==df['Lower Band'].iloc[i-1] and df['Close 5'].iloc[i]>=df['Lower Band'].iloc[i]:
    df['SuperTrend'].iloc[i]=df['Lower Band'].iloc[i]
  elif df['SuperTrend'].iloc[i-1]==df['Lower Band'].iloc[i-1] and df['Close 5'].iloc[i]<=df['Lower Band'].iloc[i]:
    df['SuperTrend'].iloc[i]=df['Upper Band'].iloc[i]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#entry price
#long entry
df['long_entry_buy']=np.nan
for i in range(n,len(df)):
  if df['SuperTrend'].iloc[n-1] < df['Close 5'].iloc[n-1] and df['SuperTrend'].iloc[n-1] < df['Open 5'].iloc[n-1] and df['High 5'].iloc[n-1] < df['Open 5'].iloc[n] and df['High 5'].iloc[n-1] < df['Close 5'].iloc[n]:
    df['long_entry_buy'].iloc[n]=df['High 5'].iloc[n-1]

for i in range(n,len(df)):
  if df['SuperTrend'].iloc[i-1] < df['Close 5'].iloc[i-1] and df['SuperTrend'].iloc[i-1] < df['Open 5'].iloc[i-1] and df['High 5'].iloc[i-1] < df['Open 5'].iloc[i] and df['High 5'].iloc[i-1] < df['Close 5'].iloc[i]:
    df['long_entry_buy'].iloc[i]=df['High 5'].iloc[i-1]

#short entry
df['short_entry_buy']=np.nan
for i in range(n,len(df)):
  if df['SuperTrend'].iloc[n-1] > df['Close 5'].iloc[n-1] and df['SuperTrend'].iloc[n-1] > df['Open 5'].iloc[n-1] and df['Low 5'].iloc[n-1] > df['Open 5'].iloc[n] and df['Low 5'].iloc[n-1] > df['Close 5'].iloc[n]:
    df['short_entry_buy'].iloc[n]=df['High 5'].iloc[n-1]

for i in range(n,len(df)):
  if df['SuperTrend'].iloc[i-1] > df['Close 5'].iloc[i-1] and df['SuperTrend'].iloc[i-1] > df['Open 5'].iloc[i-1] and df['Low 5'].iloc[i-1] > df['Open 5'].iloc[i] and df['Low 5'].iloc[i-1] > df['Close 5'].iloc[i]:
    df['short_entry_buy'].iloc[i]=df['High 5'].iloc[i-1]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
from datetime import datetime
def convert_datetime(dt):
    return datetime.strftime(dt, '%Y-%m-%d %H:%M-%S')

df['date1']= df ['Date'].apply(convert_datetime)

df['date2'] = df['date1'].apply(lambda x : x[:-8])

df['date'] = df['date2'].astype(str)+' '+df['Time'].astype(str)



df['Date'] = pd.to_datetime(df['date'])
df=df.drop(['date', 'date2', 'date1'], axis = 1)

df2= df.copy()

#df.set_index(df['Date'], inplace=True)


In [ ]:
df.head()

,Ticker,Date,Final Date format,Time,Open 5,High 5,Low 5,Close 5,Volume 5,Year,atr_18_SMA,Upper Basic,Lower Basic,Upper Band,Lower Band,SuperTrend,long_entry_buy,short_entry_buy
0,BANKNIFTY_F1,2015-01-01 09:20:00,20150101,09:20:00,18861.35,18867.95,18793.0,18808.35,84825,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BANKNIFTY_F1,2015-01-01 09:25:00,20150101,09:25:00,18808.00,18835.00,18802.2,18829.00,40925,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BANKNIFTY_F1,2015-01-01 09:30:00,20150101,09:30:00,18826.95,18833.40,18820.0,18826.10,27000,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BANKNIFTY_F1,2015-01-01 09:35:00,20150101,09:35:00,18825.05,18829.10,18810.0,18810.00,18025,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BANKNIFTY_F1,2015-01-01 09:40:00,20150101,09:40:00,18805.00,18808.95,18775.0,18803.95,60100,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91737 entries, 0 to 91736
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Ticker             91737 non-null  object        
 1   Date               91737 non-null  datetime64[ns]
 2   Final Date format  91737 non-null  int64         
 3   Time               91737 non-null  object        
 4   Open 5             91737 non-null  float64       
 5   High 5             91737 non-null  float64       
 6   Low 5              91737 non-null  float64       
 7   Close 5            91737 non-null  float64       
 8   Volume 5           91737 non-null  int64         
 9   Year               91737 non-null  int64         
 10  atr_18_SMA         91719 non-null  float64       
 11  Upper Basic        91719 non-null  float64       
 12  Lower Basic        91719 non-null  float64       
 13  Upper Band         91719 non-null  float64       
 14  Lower 

In [ ]:
df['Date'].iloc[n-1].date()
df['Date'].iloc[n-1].time()
df['Date'].iloc[n-1].time() > datetime.time(9,30)
df['Date'].iloc[n-1].time() < datetime.time(15,0)